In [3]:
import numpy as np
import numpy
import pandas as pd
from sklearn.preprocessing import StandardScaler
import math

###############***********************KNN简单实现****************###############
def KNN(test,dataset,label,k):
    #计算欧氏距离
    dist_ = test - dataset
    dist_2 = dist_**2#此处相当于(a,b)-(c,d) = (a-c,b-d)---->((a-c)**2,(b-d)**2)
    #按行相加
    dist_sum = np.sum(dist_2,axis=1)#得到一个列表
    dist = dist_sum**0.5#得到测试点与每个样本点的欧式距离
    #将距离从小到达进行排序
    dist_sort = np.argsort(dist)#返回索引值
    print(type(dist_sort))
    print(dist_sort)
    #选取最近的k个距离
    ls = []
    for i in range(k):
        ls.append(label[dist_sort[i]])#拿到该索引值对应的标签
        
    a=0
    b=0
    c=0
    for i in range(k):
        if ls[i] == 0:
            a+=1
        elif ls[i] == 1:
            b+=1
        elif ls[i] == 2:
            c+=1
    print(a)
    print(b)
    print(c)
    if a>b and a>c:
        labels = 0
    
    elif b>a and b>c:
        labels = 1
    elif c>a and c>b:
        labels = 2
    
    return labels
    '''
    classCount={}
    for i in range(k):
        voteLabel = label[dist_sort[i]]
        ###对选取的K个样本所属的类别个数进行统计
        classCount[voteLabel] = classCount.get(voteLabel,0) + 1
    maxCount = 0
    for key,value in classCount.items():
        if value > maxCount:
            maxCount = value
            classes = key
    '''

In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['label'] = iris.target
df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
data = np.array(df.iloc[:, [0, 1, -1]])#此处如果没有转换为np.array则会报错
X, y = data[:,:-1], data[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [5]:
##**************KNN一般实现*******************##
class KNN:
    def __init__(self, X_train, y_train, n_neighbors, p):
        self.n = n_neighbors
        self.p = p#可以用来选择距离度量的方式
        self.X_train = X_train
        self.y_train = y_train
    
    def predict(self, X):
        # 取出n个点
        label_list = []
        for d in range(len(X)):
            knn_list = []
            for i in range(self.n):
                dist = np.linalg.norm(X[d] - self.X_train[i], ord=self.p)#求2范数,即欧氏距离
                knn_list.append((dist, self.y_train[i]))

            for i in range(self.n, len(self.X_train)):
                max_index = knn_list.index(max(knn_list, key=lambda x: x[0]))
                dist = np.linalg.norm(X[d] - self.X_train[i], ord=self.p)
                if knn_list[max_index][0] > dist:
                    knn_list[max_index] = (dist, self.y_train[i])
            #上面两个循环可以找出距离测试点最近的k个点
            # 统计
            knn = [k[-1] for k in knn_list]#把最小的距离对应的样本的类别放到knn列表
            count_pairs = Counter(knn)#查看每个类别出现的频数
            #max_count = sorted(count_pairs, key=lambda x:x)[-1]该语句应该找不出出现次数最多的类别把
            count_pair_dict = dict(count_pairs)
            max_count = max(count_pair_dict,key = count_pair_dict.get)
            label_list.append(max_count)
        return label_list
    
    def score(self, y_pred, y_test):
        right_count = 0
        for y_pre, y in zip(y_pred, y_test):
            if y_pre == y:
                right_count += 1
        right_rate = right_count/len(y_test)
        return right_rate

In [6]:
knn = KNN(X_train,y_train,n_neighbors=5,p=2)
y_pre = np.array(knn.predict(X_test))
y_pre

array([0., 2., 1., 1., 0., 1., 2., 0., 0., 0., 1., 1., 0., 0., 2., 1., 2.,
       2., 1., 2., 0., 1., 0., 0., 1., 0., 2., 0., 1., 0., 1., 2., 2., 1.,
       2., 1., 0., 1., 2., 1., 1., 0., 1., 2., 1.])

In [7]:
knn.score(y_pre,y_test)

0.7111111111111111

#############**********KD_Tree实现快速搜索*********############

def createKDTree(data_list):
    #找出方差最大的维度
    dimension = len(data_list[0])
    #var_list = []#用来存放维度对应的方差
    max_var = 0
    for i in range(dimension):
        dim_list = []#用来存放每一个维度上的值
        for d in data_list:#第一次循环，第一个元素的第一个维度上面的值
            dim_list.append(d[i])#得到了所欲元素中第一个维度上值的列表
        #var1 = np.var((i,dim_list))这样做不好做比较
        var = np.var(dim_list)
        #print(dim_list)
        if var>max_var:
            max_var = var
            first_split_dim = i
            sort_list = np.sort(dim_list)
        #print(max_var)
        #print(sort_list)
        data_list.sort(key=lambda x: x[first_split_dim])#根据first_split_dim维度排序
    return data_list,first_split_dim

In [8]:
class KD_node:  
    def __init__(self, point=None, split=None, LL = None, RR = None):  
        """ 
        point:数据点 
        split:划分域 
        LL, RR:节点的左儿子跟右儿子 
        """  
        self.point = point  
        self.split = split  
        self.left = LL  
        self.right = RR  

In [9]:
def createKDTree(root,data_list):  
    """ 
    root:当前树的根节点 
    data_list:数据点的集合(无序) 
    return:构造的KDTree的树根 
    """  
    LEN = len(data_list)  
    if LEN == 0:  
        return None 
    #数据点的维度  
    dimension = len(data_list[0])  
    #方差  
    max_var = 0  
    #最后选择的划分域  
    split = 0;  
    for i in range(dimension):  
        ll = []  
        for t in data_list:  
            ll.append(t[i])  
        var = np.var(ll)
        #print(var)
        if var > max_var:  
            max_var = var  
            split = i
            #print(i)
    print('每次分割的维度',split)
    #根据划分域的数据对数据点进行排序  
    data_list.sort(key=lambda x: x[split])  
    #print(data_list)
    #选择下标为len / 2的点作为分割点  
    point = data_list[LEN // 2] 

    root = KD_node(point, split) 
    #print(root.point)
    #递归调用函数本身
    #print(data_list)
    root.left = createKDTree(root.left,data_list[0:(LEN // 2)])
    #将[[2,3]]判定后，此时data_list = [[2,3]] len = 1 // 2 =0 data_list[0:0] = []
    #进行判断len时 就直接return
    #LEN = len(data_list)  
    #if LEN == 0:  
        #return
    #print(data_list)
    root.right = createKDTree(root.right,data_list[(LEN // 2 + 1):LEN])  
    #print(root.left)
    #print(root.right)
    #print(root.point)
    return root #root是一个类对象，因此返回也是一个类对象 

In [10]:
x_test=[[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]]
root = KD_node()

In [11]:
root1 = createKDTree(root,x_test)

每次分割的维度 0
每次分割的维度 1
每次分割的维度 0
每次分割的维度 0
每次分割的维度 1
每次分割的维度 0


In [12]:
def preorder(root):  
    """ 
    KDTree的前序遍历 
    """  
    print(root.point)  
    if root.left:  
        preorder(root.left)  
    if root.right:  
        preorder(root.right)   
preorder(root1)

[7, 2]
[5, 4]
[2, 3]
[4, 7]
[9, 6]
[8, 1]


In [19]:
def searchNN(root,test):
    '''
    root:表示节点
    test:表示测试点
    '''
    #将root初始化为根节点开始判断
    point = root.point#得到节点对应的值
    print(point)
    v = list(map(lambda x:x[0]-x[1],zip(point,test)))
    min_dist = np.linalg.norm(v,ord=2)#计算距离
    #第一步二分叉树查找
    '''
    从根节点开始查找，知道叶子节点，记录最小距离和对应的节点值，并记录搜寻过程
    '''
    nod_list = []#记录搜索节点路径
    nod_point_list = []
    temp_root = root#先从根节点开始
    a = 0
    while temp_root:
        nod_list.append(temp_root)
        nod_point_list.append(temp_root.point)
        temp_point = temp_root.point
        print(nod_point_list)
        v = list(map(lambda x:x[0]-x[1],zip(temp_point,test)))
        d = np.linalg.norm(v,ord = 2)
        a+=1
        if d<min_dist:
            min_dist =d
            point = temp_point
        ss = temp_root.split
        print('该节点的分割维度',ss)
        if test[ss]<temp_point[ss]:
            temp_root = temp_root.left
            
        else:
            temp_root = temp_root.right
            
        if a==3:
            print('打印该节点信息',temp_root)
        print(a)
    #第二步回溯查找
    '''
    回溯查找就是递归的往回退，查找该子节点的父节点对应的区域是否还有实例\
    与测试点的距离比当前距离更小
    '''
    print('**********************************************************')
    while nod_list:
        #从后往前回退遍历该列表
        back_point = nod_list.pop()#可以得到列表的最后一个元素，将钙元素从列表‘删除’
        bs = back_point.split
        print(bs)
        print('查看该结点对应的实例',back_point.point)
        if np.abs(test[bs]-back_point.point[bs]) < min_dist:
            print(test[bs],back_point.point[bs])
            if test[bs] <= back_point.point[bs]:
                temp_root = back_point.right
            else:
                temp_root = back_point.left
            print('回溯查找时，查看结点信息',temp_root)
        if temp_root:
            nod_list.append(temp_root)##?？？？
            v = list(map(lambda x:x[0]-x[1],zip(temp_root.point,test)))
            current_dist = np.linalg.norm(v,ord = 2)
            if current_dist<min_dist:
                min_dist = current_dist
                point = temp_root.point
        print(min_dist)
    return min_dist,point

In [21]:
query = [2.5,4.5]
searchNN(root1,query)

[7, 2]
[[7, 2]]
该节点的分割维度 0
1
[[7, 2], [5, 4]]
该节点的分割维度 1
2
[[7, 2], [5, 4], [4, 7]]
该节点的分割维度 0
打印该节点信息 None
3
**********************************************************
0
查看该结点对应的实例 [4, 7]
2.5 4
回溯查找时，查看结点信息 None
2.5495097567963922
1
查看该结点对应的实例 [5, 4]
4.5 4
回溯查找时，查看结点信息 <__main__.KD_node object at 0x0000000009302518>
1.5811388300841898
0
查看该结点对应的实例 [2, 3]
2.5 2
回溯查找时，查看结点信息 None
1.5811388300841898
0
查看该结点对应的实例 [7, 2]
1.5811388300841898


(1.5811388300841898, [2, 3])